In [4]:
%load_ext autoreload
%autoreload 2
import jieba
import transformers as ts
import pandas as pd
import numpy as np
import torch
import json
from sklearn.model_selection import train_test_split
from ark_nlp.model.ner.global_pointer_bert import Dataset as arkData
from ark_nlp.model.ner.global_pointer_bert import Tokenizer,GlobalPointerBertConfig,GlobalPointerBert,get_default_model_optimizer,Task,Predictor
from torch.utils.data import DataLoader,Dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_test=pd.read_json('./datasets/train.json')
lens=len(df_test)
df_test.head(3)

,text,ner_list,type,doc_id
0,血管内皮生长因子表达水平与宫颈癌患者预后的关系,"[{'mention': '血管内皮生长因子表达水平与宫颈癌患者预后的关系', 'label...",Title,2500
1,目的探讨地尔硫卓在治疗扩张型心肌病中的临床疗效。方法将2014年9月至2017年12月在红河...,"[{'mention': '探讨地尔硫卓在治疗扩张型心肌病中的临床疗效', 'label':...",Abstract,2501
2,地尔硫卓在治疗扩张型心肌病中的临床疗效观察,"[{'mention': '地尔硫卓在治疗扩张型心肌病中的临床疗效观察', 'label':...",Title,2501


In [14]:
#用于W2NER
result=[]
for i in range(0,lens):
    if df_test.loc[i,'type'] == 'Title':
        text=df_test.loc[i,'text'].replace(' ','')
        #token
        sentence=[char for char in text]
        #实体
        ner=[]
        for n in df_test.loc[i,'ner_list']:
            entity=n['mention'].replace(' ','')
            l=text.index(entity)
            r=l+len(entity)
            index=[j for j in range(l,r)]
            type=n['label']
            ner.append({
                'index':index,
                'type':type,
            })
        #分词
        word=[]
        split_word=jieba.tokenize(text)
        for w in split_word:
            widx=[j for j in range(w[1],w[2])]
            word.append(widx)
        #组装
        result.append({
            'sentence':sentence,
            'ner':ner,
            'word':word,
        })
result_json=json.dumps(result,ensure_ascii = False)
with open('./datasets/w2ner/title_train.json','w',encoding='utf-8')as f:
    f.write(result_json)
    f.close()

In [ ]:
bigcat=[]
smallcat={
    'C':[],
    'I':[],
    'O':[],
    'P':[],
    'S':[],
}
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            newbig={
                'text':ner['mention'],
                'label':l,
            }
            bigcat.append(newbig)
            smallcat[l].append({
                'text':ner['mention'],
                'label':ner['label'][1:],
            })
    else:
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            curtext=ner['mention']
            if len(curtext) > 200:
                todel=len(curtext)-202
                l=todel//2
                r=todel-l
                curtext=curtext[l:-r]
            newbig={
                'text':curtext,
                'label':l,
            }
            bigcat.append(newbig)
            smallcat[l].append({
                'text':curtext,
                'label':ner['label'][1:],
            })
big_json=1